<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_two_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 여러 상담원 및 일반 사용자와의 협업 작업 해결

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 툴 또는 사람으로 구동되는 대화형 에이전트를 제공합니다. 이 프레임워크는 다중 에이전트 대화를 통해 툴 사용과 사람 참여가 가능합니다. 이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

이 노트북에서는 여러 에이전트와 사람이 함께 작업하여 작업을 수행하는 애플리케이션을 시연합니다. 'AssistantAgent'는 사용자가 주어진 작업을 실행하기 위해 Python 코드를 작성할 수 있는 LLM 기반 에이전트입니다(Python 코딩 블록에서). UserProxyAgent`는 사용자가 `AssistantAgent`가 작성한 코드를 실행할 수 있도록 프록시 역할을 하는 에이전트입니다. 서로 다른 인간 사용자를 나타낼 수 있는 여러 개의 `UserProxyAgent` 인스턴스를 생성합니다.

요구사항 ## 요구사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 설치하세요:
```bash
pip 설치 pyautogen
```

In [1]:
# %pip install pyautogen~=0.1.1

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.

먼저 지정된 이름의 환경 변수(이 예제에서는 "OAI_CONFIG_LIST")를 찾으며, 이 환경 변수는 유효한 json 문자열이어야 합니다. 해당 변수를 찾을 수 없으면 같은 이름의 json 파일을 찾습니다. 모델별로 구성을 필터링합니다(다른 키로도 필터링할 수 있음).

json은 다음과 같습니다:
```json
[
    {
        "모델": "gpt-4",
        "api_key": "<귀하의 OpenAI API 키는 여기>"
    },
    {
        "model": "gpt-4",
        "api_key": "<귀하의 Azure OpenAI API 키는 여기>",
        "api_base": "<귀하의 Azure OpenAI API 베이스는 여기>",
        "api_type": "azure",
        "api_version": "2023-07-01-preview"
    },
    {
        "model": "GPT-4-32K",
        "api_key": "<귀하의 Azure OpenAI API 키는 여기>",
        "api_base": "<귀하의 Azure OpenAI API 베이스는 여기>",
        "api_type": "azure",
        "api_version": "2023-07-01-preview"
    }
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 '파일 업로드' 아이콘을 선택해 파일을 업로드할 수 있습니다.


In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

## 에이전트 생성

두 상담원 간의 대화를 시작하고 결과 요약을 반환하는 `ask_expert` 함수를 정의합니다. "assistant_for_expert"라는 이름의 어시스턴트 에이전트와 "expert"라는 이름의 사용자 프록시 에이전트를 구성합니다. 사용자 프록시 에이전트에서 `human_input_mode`를 "항상"으로 지정하여 항상 전문가 사용자에게 피드백을 요청합니다.

In [3]:
def ask_expert(message):
    assistant_for_expert = autogen.AssistantAgent(
        name="assistant_for_expert",
        llm_config={
            "temperature": 0,
            "config_list": config_list,
        },
    )
    expert = autogen.UserProxyAgent(
        name="expert",
        human_input_mode="ALWAYS",
        code_execution_config={"work_dir": "expert"},
    )

    expert.initiate_chat(assistant_for_expert, message=message)
    expert.stop_reply_at_receive(assistant_for_expert)
    # expert.human_input_mode, expert.max_consecutive_auto_reply = "NEVER", 0
    # final message sent from the expert
    expert.send("summarize the solution and explain the answer in an easy-to-understand way", assistant_for_expert)
    # return the last message the expert received
    return expert.last_message()["content"]


"assistant_for_student"라는 이름의 또 다른 어시스턴트 에이전트와 "student"라는 이름의 사용자 프록시 에이전트를 구성합니다. 사용자 프록시 에이전트에서 `human_input_mode`를 "종료"로 지정하면 어시스턴트 에이전트로부터 "종료" 신호를 받으면 피드백을 요청합니다. 생성한 `ask_expert` 함수를 사용하도록 `AssistantAgent`의 `functions`와 `UserProxyAgent`의 `function_map`을 설정합니다.

간단하게 하기 위해 `ask_expert` 함수는 로컬에서 실행되도록 정의했습니다. 실제 애플리케이션의 경우 이 함수는 전문가 사용자와 상호 작용하기 위해 원격으로 실행되어야 합니다.

In [4]:
assistant_for_student = autogen.AssistantAgent(
    name="assistant_for_student",
    system_message="You are a helpful assistant. Reply TERMINATE when the task is done.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        # Excluding azure openai endpoints from the config list.
        # Change to `exclude="openai"` to exclude openai endpoints, or remove the `exclude` argument to include both.
        "config_list": autogen.config_list_openai_aoai(exclude="aoai"),
        "model": "gpt-4-0613",  # make sure the endpoint you use supports the model
        "temperature": 0,
        "functions": [
            {
                "name": "ask_expert",
                "description": "ask expert when you can't solve the problem satisfactorily.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask expert. Ensure the question includes enough context, such as the code and the execution result. The expert does not know the conversation between you and the user unless you share the conversation with the expert.",
                        },
                    },
                    "required": ["message"],
                },
            }
        ],
    }
)

student = autogen.UserProxyAgent(
    name="student",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "student"},
    function_map={"ask_expert": ask_expert},
)

## 작업 수행

학생 프록시 에이전트의 `initiate_chat()` 메서드를 호출하여 대화를 시작합니다. 아래 셀을 실행하면 학생 대리 상담원이 메시지 끝에 "종료" 신호를 보낸 후 피드백을 제공하라는 메시지가 표시됩니다. 피드백을 제공하지 않으면(직접 Enter 키를 눌러) 대화가 종료됩니다. "종료" 신호가 전송되기 전에 학생 프록시 상담원은 사용자를 대신하여 보조 상담원이 제안한 코드를 실행하려고 시도합니다.

In [5]:
# the assistant receives a message from the student, which contains the task description
student.initiate_chat(
    assistant_for_student,
    message="""Find $a + b + c$, given that $x+y \\neq -1$ and 
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
""",
)

student (to assistant_for_student):

Find $a + b + c$, given that $x+y \neq -1$ and 
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant_for_student (to student):

To find the values of $a$, $b$, and $c$, we need to solve the system of equations. However, the system of equations you provided seems to be incomplete or incorrect. The equations are not properly formatted, and it's unclear how $x$ and $y$ are related to $a$, $b$, and $c$. 

Could you please provide the correct system of equations?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
student (to assistant_for_student):



--------------------------------------------------------------------------------
assistant_for_student (to student):

***** Suggested function Call: ask_expert *****
Arguments: 
{
  "message": "The user pr

어시스턴트가 전문가와 상담이 필요한 경우 `ask_expert`에 대한 함수 호출을 제안합니다. 이 경우 다음과 같은 줄이 표시됩니다:

***** 제안된 함수 호출: ask_expert *****
